In [ ]:
# imports as per

import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# variable declaration

nx = 41
ny = 41
nt = 500

nit = 50

c = 1

dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
dt = 0.001

x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)

X, Y = numpy.meshgrid(x, y)

rho = 1
nu = 0.1

u = numpy.zeros((ny, nx))
v = numpy.zeros((ny, nx))
p = numpy.zeros((ny, nx))
b = numpy.zeros((ny, nx))

In [ ]:
# define a function that outputs the terms in the square brackets in the pressure equations

def build_b(dx, dy, dt, u, v, rho, b):
    
    b[1:-1, 1:-1] = ((rho * dx ** 2 * dy ** 2) / (2 * (dx ** 2 + dy ** 2)) 
                     
                     * ((1 / dt) * ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx) + (v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy)
                                    
                                    - ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx)) ** 2
                                    
                                    - 2 * (u[2:, 1:-1] - u[0:-2, 1:-1]) / (2 * dy) * (v[1:-1, 2:] - v[1:-1, 0:-2]) / (2 * dx)
                                    
                                    - ((v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy)) ** 2)))
    
    return b

In [1]:
# iterative function to get the pressure field

def pressure_poission(dx, dy, b, p):
    
    pn = numpy.empty_like(p)
    
    for n in range(nit):
        
        pn = p.copy()     
        
        p[1:-1, 1:-1] = (((pn[1:-1, 2:] - pn[1:-1, 0:-2]) * dy ** 2 + 
                         
                         (pn[2:, 1:-1] - pn[0:-2, 1:-1]) * dx ** 2) / (2 * (dx ** 2 + dy ** 2)) -
                        
                        b[1:-1, 1:-1])
        
        p[-1, :] = 0 # p = 0 at y = 0
        p[0, :] = p[1, :] # dp/dy = 0 at y = 0
        p[:, 0] = p[:, 1] # dp/dx = 0 at x = 0
        p[:, -1] = p[:, -2] # dp/dx = 0 at x = 2
        
        return p

In [ ]:
def build_u(dx, dy, dt, u, v, p, nu):
    
    u[1:-1, 1:-1] = (u[1:-1, 1:-1] - u[1:-1, 1:-1] * (dt / dx) * (u[1:-1, 1:-1] - u[1:-1, 0:-2]) -
                     
                                     v[1:-1, 1:-1] * (dt / dy) * (u[1:-1, 1:-1] - u[0:-2, 1:-1]) -
                    
                                     (dt / (2 * rho * dx)) * (p[1:-1, 2:] - p[1:-1, 0:-2]) +
                    
                    nu * ( (dt / dx ** 2) * (u[1:-1, 2:] - u[1:-1, 1:-1] + u[1:-1, 0:-2]) +
                          
                           (dt / dy ** 2) * (u[2: , 1:-1] - u[1:-1, 1:-1] + u[0:-2, 1:-1])))